In [1]:
from config import *
from data_preprocessing import *
from utils import *
from models import *
from train import *
from evaluate import *

import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

set_seeds(RANDOM_SEED)

In [2]:
df = load_and_prepare_data(DATA_PATH)

print(df.shape)
print(df.head())

(2880844, 20)
   weekday    server_timestamp  voltage  current  power  frequency  energy  \
0        3 2021-08-05 12:44:50    119.9     1.09  118.7       60.0  171.89   
1        3 2021-08-05 12:44:51    119.9     1.05  121.3       60.0  171.89   
2        3 2021-08-05 12:44:52    119.9     1.01  117.5       59.9  171.89   
3        3 2021-08-05 12:44:53    119.9     1.02  119.2       59.9  171.89   
4        3 2021-08-05 12:44:54    120.0     0.91  106.5       59.9  171.89   

   power_factor  sensor_temperature  cpu_usage_percent  WORKSTATION_CPU_POWER  \
0          0.90               33.33                0.0                    0.0   
1          0.97               33.89                0.0                    0.0   
2          0.97               33.33                0.0                    0.0   
3          0.97               33.33                0.0                    0.0   
4          0.97               33.33                0.0                    0.0   

   WORKSTATION_CPU_TEMP  WORKS

In [3]:
df[['server_timestamp']].head(50)


,server_timestamp
0,2021-08-05 12:44:50
1,2021-08-05 12:44:51
2,2021-08-05 12:44:52
3,2021-08-05 12:44:53
4,2021-08-05 12:44:54
5,2021-08-05 12:44:55
6,2021-08-05 12:44:56
7,2021-08-05 12:44:57
8,2021-08-05 12:44:58
9,2021-08-05 12:45:00


In [4]:
# Ensure sorted (important)
df = df.sort_values("server_timestamp").reset_index(drop=True)

# Compute time differences between consecutive rows
time_deltas = df["server_timestamp"].diff().dropna()

# Convert to seconds
time_deltas_seconds = time_deltas.dt.total_seconds()

# Basic statistics
print("Sampling interval statistics (seconds):")
print(time_deltas_seconds.describe())


Sampling interval statistics (seconds):
count    2.880843e+06
mean     3.623385e+00
std      3.366618e+03
min      0.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      5.683034e+06
Name: server_timestamp, dtype: float64
